# Examen ETL: SPARK 12+1/02

## Realizado por Daniel Tomé Gordo

Se podrá utilizar toda la información que se encuentra en el campus. 

Se va a trabajar sobre varios ficheros de datos:

Usuarios: id_usuario::sexo::edad::id_profesion::codigo_postal

Peliculas: id_pelicula::titulo (año)::tipo1|tipo2|tipo3

Ratings: id_pelicula::id_usuario::puntuacion::fecha_timestamp

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

In [1]:
# Nota: Carga de las puntuaciones
# Función para parsear la fecha

from datetime import datetime
dateparse = lambda x: datetime.fromtimestamp(float(x))

## Primera tarea: Inicializar spark context y cargar los datos desde los ficheros.

In [2]:
# Se importa la librería y se carga el contexto Spark
from pyspark import SparkContext

sc = SparkContext()

In [35]:
# Se cargan los cuatro archivos de datos

datos_UserRatings = sc.textFile('./user_ratings.txt')
datos_Movies = sc.textFile('./movies.dat.txt')
datos_Ratings = sc.textFile('./ratings.dat.txt')
datos_Users = sc.textFile('./users.dat.txt')

In [36]:
datos_UserRatings_split = datos_UserRatings.map(lambda x: x.split('::'))
datos_Movies_split = datos_Movies.map(lambda x: x.replace('|', '::')).map(lambda x: x.split('::'))
datos_Ratings_split = datos_Ratings.map(lambda x: x.split('::'))
datos_Users_split = datos_Users.map(lambda x: x.split('::'))

In [85]:
datos_Ratings_split.take(1)

[['1', '1193', '5', '978300760']]

In [4]:
# Parseo de la primera base de datos (Ratings). Lo primero es separar por :: luego se crea un Row para tener los campos identificados

from pyspark.sql import Row
datos_Ratings_row = datos_Ratings.map(lambda x: x.split('::')).\
    map(lambda x: Row(Usuario = x[0],
                     Pelicula = x[1],
                     Puntuacion = x[2],
                     Fecha = dateparse(x[3])))
datos_Ratings_row.take(1)

[Row(Fecha=datetime.datetime(2000, 12, 31, 22, 12, 40), Pelicula='1193', Puntuacion='5', Usuario='1')]

In [18]:
datos_Movies_row = datos_Movies.map(lambda x: x.replace('|', '::')).\
    map(lambda x: x.split('::')).\
    map(lambda x: Row(IDPelicula = x[0],
                     Titulo = x[1],
                     Ano = str(x[1][-6:]),
                     Genero1 = x[2],
                     Genero2 = x[3],
                     Genero3 = x[4]))
    
datos_Movies_row.take(1)

[Row(Ano='(1995)', Genero1='Animation', Genero2="Children's", Genero3='Comedy', IDPelicula='1', Titulo='Toy Story (1995)')]

In [32]:
datos_UserRatings_row = datos_UserRatings.map(lambda x: x.split('::')).\
    map(lambda x: Row(IDUsuario = x[0],
                      Sexo = x[1],
                     Edad = int(x[2]),
                     CodigoPostal = int(x[3]),
                     Profesion = x[4],
                     IDPelicula = x[5],
                     Rating = x[6],
                     Fecha = dateparse(x[7])))
    
datos_UserRatings_row.take(1)

[Row(CodigoPostal=10, Edad=1, Fecha=datetime.datetime(2000, 12, 31, 22, 12, 40), IDPelicula='1193', IDUsuario='1', Profesion='48067', Rating='5', Sexo='F')]

In [31]:
datos_UserRatings.take(1)

['1::F::1::10::48067::1193::5::978300760']

## Segunda tarea: Media de puntuaciones globales por año. ¿Hay algún año significativamente distinto? 

En primer lugar habrá que realizar una agrupación por el año y luego realizar una media entre los registros que tengan un mismo año. Para ello, el primer paso será hallar el año y la valoración. El formato será clave-valor siendo la clave el año y el valor una combinación del rating y un conteo. El segundo paso será reducir por la clave, sumando tanto los ratings como el conteo creado. De esta manera, se tendrá un formato del tipo (Año, SumaRating, SumaConteo). El último paso de todos será realizar la media por año, que se puede realizar de manera sencilla dividiendo el primer valor de la clave (SumaRating) entre el segundo valor de la clave(SumaConteo). Así se tendrá cada año con la media de sus películas

In [96]:
media_año = datos_UserRatings_split.map(lambda x: (dateparse(x[7]).year, (int(x[6]), 1)))

In [98]:
media_año2 = media_año.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])).persist()

In [100]:
resultado = media_año2.map(lambda x: (x[0], x[1][0] / x[1][1])).sortBy(lambda x: x[1], ascending = False)

In [102]:
resultado.collect()

[(2000, 3.5903916742285498),
 (2001, 3.5122542537247643),
 (2003, 3.486559139784946),
 (2002, 3.458828911253431)]

## Tercera pregunta: ¿Cuál es la película más votada por los mayores de 45? 

Para este ejercicio el primer paso será filtrar por todas aquellas películas que hayan sido vistas por personas mayores de 45 años. Se sumarán todos los rating y el que mayor número de votos tenga será el resultante

In [108]:
mayores = datos_UserRatings_split.filter(lambda x: int(x[2]) >= 45).\
    map(lambda x: (x[5], int(x[6])))
    
mayores2 = mayores.reduceByKey(lambda a, b: a + b)

In [110]:
mayores_res = mayores2.sortBy(lambda x: x[1], ascending = False).take(1)

In [111]:
print ('La película con más valoracion es el ID', mayores_res[0][0])

La película con más valoracion es el ID 2858


## Cuarta pregunta: ¿Cuál es la puntuación media de las peliculas de acción del año 2000?

Lo primero será filtran por las películas del año 2000 y luego las que sean del género de acción. Tras esto sólo faltará realizar la media de estas películas. Para este ejercicio es necesario realizar una unión entre los datasets Movies y el dataset Ratings, ya que en el dataset movies, no se encuentra toda la información. Para el join sólo será necesario usar de la tabla ratings el ID de la película y el rating dado.

Una vez que ambos datasets estén unidos sólo queda filtrar por el género, el año y finalmente calcular la valoración media.

## Quinta pregunta: ¿ Cuál es el año en que mayor número de usuarios votaron?

En este caso se necesita sólo el dataset UserRatings ya que se sacará el año y se llevará un conteo en el valor del formato.
El año con un mayor número de usuarios que votaron fue el año 2000.

In [121]:
mayor = datos_UserRatings_split.map(lambda x: (dateparse(x[7]).year, 1)).\
    reduceByKey(lambda a, b: a + b).persist()

In [124]:
mayor.collect()

[(2000, 904757), (2002, 24046), (2001, 68058), (2003, 3348)]

## Sexta pregunta: ¿ Cuál es la película con mejor puntación media?

Para esta pregunta será necesario unir los datasets películas y rating, por la categoría id_peliculas que ambos tienen.